Step 1. Import library

In [15]:
import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np

# Built In Imports

from datetime import datetime
from glob import glob
import warnings
import IPython
import urllib
import zipfile
import pickle
import shutil
import string
import math
import tqdm
import time
import os
import gc
import re

# Visualization Imports

from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from PIL import Image
import matplotlib
import plotly
import PIL
import cv2

from pycocotools.coco import COCO

# Other Imports

from tqdm.notebook import tqdm

LABEL_COLORS = [px.colors.label_rgb(px.colors.convert_to_RGB_255(x)) for x in sns.color_palette("Spectral", 10)]
LABEL_COLORS_WOUT_NO_FINDING = LABEL_COLORS[:8]+LABEL_COLORS[9:]

Step 2. Convert json to dataframe

In [16]:
coco = COCO('/root/dataset/train.json')
train_df = pd.DataFrame()
image_ids = []
class_name = []
class_id = []
x_min = []
y_min = []
x_max = []
y_max = []
classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

for image_id in coco.getImgIds():
    imageinfo = coco.loadImgs(image_id)[0]
    annids = coco.getAnnIds(imgIds=imageinfo['id'])
    anns = coco.loadAnns(annids)
    filename = imageinfo['file_name']

    for ann in anns:
        image_ids.append(filename)
        class_name.append(classes[ann['category_id']])
        class_id.append(ann['category_id'])
        x_min.append(float(ann['bbox'][0]))
        y_min.append(float(ann['bbox'][1]))
        x_max.append(float(ann['bbox'][0]) + float(ann['bbox'][2]))
        y_max.append(float(ann['bbox'][1]) + float(ann['bbox'][3]))

train_df['image_id'] = image_ids
train_df['class_name'] = class_name
train_df['class_id'] = class_id
train_df['x_min'] = x_min
train_df['y_min'] = y_min
train_df['x_max'] = x_max
train_df['y_max'] = y_max

Exception ignored in: <function tqdm.__del__ at 0x7fea1bd51ee0>
Traceback (most recent call last):
  File "/opt/conda/envs/level2/lib/python3.12/site-packages/tqdm/std.py", line 1149, in __del__
    self.close()
  File "/opt/conda/envs/level2/lib/python3.12/site-packages/tqdm/notebook.py", line 278, in close
    self.disp(bar_style='danger', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


Step 3. EDA Annotation Per Images : 한 이미지에 몇 개의 Bounding box가 있는지, 그 분포를 나타낸 결과입니다.

In [ ]:
fig = px.histogram(train_df.image_id.value_counts(), log_y=False, color_discrete_sequence=['indianred'], opacity=0.7,
                   labels={"value":"Number of Annotations Per Image"},
                   title="<b>DISTRIBUTION OF # OF ANNOTATIONS PER IMAGE "
                   "<i><sub>(Log Scale for Y-Axis)</sub></i></b>",)

fig.update_layout(showlegend=False,
                  xaxis_title="<b>이미지당 BBOX 개수</b>",
                  yaxis_title="<b>이미지 개수</b>",
                  width=800)

fig.show()

Unique Class Per Image

한 이미지의 몇 개의 unique한 class가 있는지, 그 분포를 나타낸 결과입니다.

In [ ]:
fig = px.histogram(train_df.groupby('image_id')["class_id"].unique().apply(lambda x: len(x)),
                   log_y=False, color_discrete_sequence=['skyblue'], opacity=0.7,
                   labels={"value":"Number of Unique class"},
                   title="<b>DISTRIBUTION OF # OF Unique Class PER IMAGE " 
                   "<i><sub>(Log Scale for Y-Axis)</sub></i></b>",
                   )

fig.update_layout(showlegend=False,
                  xaxis_title="<b>Number of Unique CLASS</b>",
                  yaxis_title="<b>Count of Unique IMAGE</b>",
                  width=800)

fig.show()

Annotations Per Class

각 Class 당 몇 개의 Annotation, 즉 bbox가 있는지, 그 분포를 나타낸 결과입니다.

In [ ]:
fig = px.bar(train_df.class_name.value_counts().sort_index(),
             color=train_df.class_name.value_counts().sort_index().index, opacity=0.85,
             color_discrete_sequence=LABEL_COLORS, log_y=False,
             labels={"y":"Annotations Per Class", "x":""},
             title="<b>Annotations Per Class</b>",)

fig.update_layout(legend_title=None,
                 xaxis_title="",
                 yaxis_title="<b>Annotations Per Class</b>",
                 width=800)

fig.show()

Object heatmap

이미지에서 bbox가 주로 어디에 위치하는지를 알아보기 위한 코드입니다.

In [ ]:
matplotlib.rcParams.update({'font.size': 16})
bbox_df = pd.DataFrame()
bbox_df['class_id'] = train_df['class_id'].values
bbox_df['class_name'] = train_df['class_name'].values
bbox_df['x_min'] = train_df['x_min'].values
bbox_df['x_max'] = train_df['x_max'].values
bbox_df['y_min'] = train_df['y_min'].values
bbox_df['y_max'] = train_df['y_max'].values
bbox_df['frac_x_min'] = train_df['x_min'].values / 1024
bbox_df['frac_x_max'] = train_df['x_max'].values / 1024
bbox_df['frac_y_min'] = train_df['y_min'].values / 1024
bbox_df['frac_y_max'] = train_df['y_max'].values / 1024
ave_src_img_height = 1024
ave_src_img_width = 1024

# DEFAULT

HEATMAP_SIZE = (ave_src_img_height, ave_src_img_width, 14)

# Initialize

heatmap = np.zeros((HEATMAP_SIZE), dtype=np.int16)
bbox_np = bbox_df[["class_id", "frac_x_min", "frac_x_max", "frac_y_min", "frac_y_max"]].to_numpy()
bbox_np[:, 1:3] *= ave_src_img_width
bbox_np[:, 3:5] *= ave_src_img_height
bbox_np = np.floor(bbox_np).astype(np.int16)

# Color map stuff

custom_cmaps = [
    matplotlib.colors.LinearSegmentedColormap.from_list(
        colors=[(0.,0.,0.), c, (0.95,0.95,0.95)],
        name=f"custom{i}") for i,c in enumerate(sns.color_palette("Spectral", 10))
    ]

custom_cmaps.pop(8) # Remove No-Finding

for row in tqdm(bbox_np, total=bbox_np.shape[0]): 
    heatmap[row[3]:row[4]+1, row[1]:row[2]+1, row[0]] += 1

fig = plt.figure(figsize=(20,25))
plt.suptitle("Heatmaps Showing Bounding Box Placement\n ", fontweight="bold", fontsize=30)

for i in range(10):
    plt.subplot(4, 4, i+1)
    if i==0:
        plt.imshow(heatmap.mean(axis=-1), cmap="bone")
        plt.title(f"Average of All Classes", fontweight="bold", fontsize=20)
        
    else:
        plt.imshow(heatmap[:, :, i-1], cmap=custom_cmaps[i-1])
        plt.title(f"{classes[i-1]} - ({i})", fontweight="bold", fontsize=20)

    plt.axis(False)

fig.tight_layout(rect=[0, 0.03, 1, 0.97])
plt.show()

In [ ]:
matplotlib.rcParams.update({'font.size': 16})
bbox_df = pd.DataFrame()
bbox_df['class_id'] = train_df['class_id'].values
bbox_df['class_name'] = train_df['class_name'].values
bbox_df['x_min'] = train_df['x_min'].values
bbox_df['x_max'] = train_df['x_max'].values
bbox_df['y_min'] = train_df['y_min'].values
bbox_df['y_max'] = train_df['y_max'].values
bbox_df['frac_x_min'] = train_df['x_min'].values / 1024
bbox_df['frac_x_max'] = train_df['x_max'].values / 1024
bbox_df['frac_y_min'] = train_df['y_min'].values / 1024
bbox_df['frac_y_max'] = train_df['y_max'].values / 1024
ave_src_img_height = 1024
ave_src_img_width = 1024

# DEFAULT

HEATMAP_SIZE = (ave_src_img_height, ave_src_img_width, 14)

# Initialize

heatmap = np.zeros((HEATMAP_SIZE), dtype=np.int16)
bbox_np = bbox_df[["class_id", "frac_x_min", "frac_x_max", "frac_y_min", "frac_y_max"]].to_numpy()
bbox_np[:, 1:3] *= ave_src_img_width
bbox_np[:, 3:5] *= ave_src_img_height
bbox_np = np.floor(bbox_np).astype(np.int16)

# Color map stuff

# 수정 부분 시작
x_coords = bbox_np[:, 1:3].reshape(-1)
half_widths = np.full(x_coords.shape[0], ave_src_img_width / 2)
mean_x = np.mean(x_coords)

if mean_x < ave_src_img_width / 2:
    bbox_np[:, 1] = np.maximum(bbox_np[:, 1], 0)
else:
    half_widths = (bbox_np[:, 2] - bbox_np[:, 1]) / 2
    half_widths = np.tile(half_widths[:, np.newaxis], (1, 2))
    bbox_np[:, 1:3] = (bbox_np[:, 1:3] - half_widths).astype(np.int16)
    bbox_np[:, 1] = np.maximum(bbox_np[:, 1], 0)
# 수정 부분 끝


for row in tqdm(bbox_np, total=bbox_np.shape[0]): 
    heatmap[row[3]:row[4]+1, row[1]:row[2]+1, row[0]] += 1

fig = plt.figure(figsize=(20,25))
plt.suptitle("Heatmaps Showing Bounding Box Placement\n ", fontweight="bold", fontsize=30)

for i in range(10):
    plt.subplot(4, 4, i+1)
    if i==0:
        plt.imshow(heatmap.mean(axis=-1), cmap="bone")
        plt.title(f"Average of All Classes", fontweight="bold", fontsize=20)
        
    else:
        plt.imshow(heatmap[:, :, i-1], cmap=custom_cmaps[i-1])
        plt.title(f"{classes[i-1]} - ({i})", fontweight="bold", fontsize=20)

    plt.axis(False)

fig.tight_layout(rect=[0, 0.03, 1, 0.97])
plt.show()

Distribution of bbox area

각 Class 별 bounding box의 크기 분포입니다.

In [ ]:
# Update bbox dataframe to make this easier

bbox_df["frac_bbox_area"] = (bbox_df["frac_x_max"]-bbox_df["frac_x_min"])*(bbox_df["frac_y_max"]-bbox_df["frac_y_min"])

# bboxdf["classidasstr"] = bboxdf["classid"].map(classes)

display(bbox_df.head())

fig = px.box(bbox_df.sort_values(by="class_name"), x="class_name", y="frac_bbox_area", color="class_name",
             color_discrete_sequence=LABEL_COLORS_WOUT_NO_FINDING, notched=True,
             labels={"class_name":"Class Name", "frac_bbox_area":"BBox Area (%)"},
             title="<b>DISTRIBUTION OF BBOX AREAS AS % OF SOURCE IMAGE AREA  "
             "<i><sub>(Some Upper Outliers Excluded For Better Visualization)</sub></i></b>")

fig.update_layout(showlegend=True,
                  yaxis_range=[0, 1.1],
                  legend_title_text=None,
                  xaxis_title="",
                  yaxis_title="<b>Bounding Box Area %</b>",
                  width=1050,
                  yaxis=dict(
                      tickmode='linear',
                      tick0=0,
                      dtick=0.2,)
)

fig.show()

Aspect Ratio for bounding boxes by class

각 Class 별 bounding box의 aspect ratio 값입니다. 이를 활용하여 anchor generator의 aspect ratio 비율을 조절해볼 수 있습니다.

In [ ]:
# Aspect Ratio is Calculated as Width/Height

bbox_df["aspect_ratio"] = (bbox_df["x_max"]-bbox_df["x_min"])/(bbox_df["y_max"]-bbox_df["y_min"])

# Display average means for each class_id so we can examine the newly created Aspect Ratio Column

display(bbox_df.groupby("class_name").mean())

# Generate the bar plot

fig = px.bar(x=sorted(classes), y=bbox_df.groupby("class_name").mean()["aspect_ratio"],
             color=sorted(classes), opacity=0.85,
             color_discrete_sequence=LABEL_COLORS_WOUT_NO_FINDING,
             labels={"x":"Class Name", "y":"Aspect Ratio (W/H)"},
             title="<b>Aspect Ratios For Bounding Boxes By Class</b>",)

fig.update_layout(
    yaxis_title="<b>Aspect Ratio (W/H)</b>",
    xaxis_title=None,
    legend_title_text=None)

fig.add_hline(y=1, line_width=2, line_dash="dot",
             annotation_font_size=13,
             annotation_text="<b>SQUARE ASPECT RATIO</b>",
             annotation_position="bottom left",
             annotation_font_color="black")

fig.add_hrect(y0=0, y1=0.5, line_width=0, fillcolor="red", opacity=0.125,
             annotation_text="<b>>2:1 VERTICAL RECTANGLE REGION</b>",
             annotation_position="bottom right",
             annotation_font_size=13,
             annotation_font_color="red")

fig.add_hrect(y0=2, y1=3.5, line_width=0, fillcolor="green", opacity=0.04,
             annotation_text="<b>>2:1 HORIZONTAL RECTANGLE REGION</b>",
             annotation_position="top right",
             annotation_font_size=13,
             annotation_font_color="green")

fig.show()